<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/%EC%BD%94%EC%8B%B8%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import re
from tqdm import tqdm
from gensim.summarization import keywords
from textblob import TextBlob

!pip install gensim


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [126]:
comic = pd.read_csv('./sample_data/Webtoon Dataset.csv')

In [127]:
# 축약형 
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "she's": "she is"}

def regularize_contractions(text):

    contractionfree_list = []
    
    for word in text.split(" "):
      if word in contractions.keys():
        contractionfree_list.append(contractions[word])
      else :
        contractionfree_list.append(word)
      
    return " ".join(contractionfree_list)


In [128]:
#defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로

def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)


# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

In [129]:
# #defining function for tokenization
# 숫자 포함 토큰화
# def tokenization(text):
#     tokens = re.split('\W+', text)

#     return tokens

# 영어만 단어단위로 토큰화
p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

In [130]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
# print(stopwords[:20]) 

#defining the function to remove stopwords from tokenized text
# stopwords 제거 
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output

In [131]:
#setting lower case
comic['Lower_Summary'] = comic['Summary'].apply(lambda x: x.lower())
comic['Lower_Summary'].head(3)

0    she's young, single and about to achieve her d...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: Lower_Summary, dtype: object

In [132]:
comic['No_Contraction_Summary'] = comic['Lower_Summary'].apply(lambda x:regularize_contractions(x))
comic['No_Contraction_Summary'].head(3)

0    she is young, single and about to achieve her ...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: No_Contraction_Summary, dtype: object

In [133]:
comic['Clean_Summary'] = comic['No_Contraction_Summary'].apply(lambda x:remove_punctuation(x))
comic['Clean_Summary'].head(3)

0    she is young  single and about to achieve her ...
1    after binge watching beauty videos online  a s...
2    after making a grisly discovery in the country...
Name: Clean_Summary, dtype: object

In [134]:
#applying function to the column
comic['Tokenied_Summary'] = comic['Clean_Summary'].apply(lambda x: tokenization(x))

comic['Tokenied_Summary'].head(3)

0    [she, is, young, single, and, about, to, achie...
1    [after, binge, watching, beauty, videos, onlin...
2    [after, making, a, grisly, discovery, in, the,...
Name: Tokenied_Summary, dtype: object

In [135]:
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))

comic['No_Stopwords_Summary'].head(3)

0    [young, single, achieve, dream, creating, incr...
1    [binge, watching, beauty, videos, online, shy,...
2    [making, grisly, discovery, countryside, small...
Name: No_Stopwords_Summary, dtype: object

In [136]:
# df1 = comic['No_Stopwords_Summary']
# doc_words_list = df1.values.tolist()

# print(doc_words_list)


def wordlist_to_string(words_list) :
  doc = []
  for doc_word in words_list :
    doc.append(doc_word)
  return " ".join(doc)


#comic['Summary_final'] = comic['Stemmed_Summary'].apply(lambda x:wordlist_to_string(x))
comic['Summary_final'] = comic['No_Stopwords_Summary'].apply(lambda x:wordlist_to_string(x))

comic['Summary_final'].head(3)

0    young single achieve dream creating incredible...
1    binge watching beauty videos online shy comic ...
2    making grisly discovery countryside small town...
Name: Summary_final, dtype: object

In [137]:
def extract_keywords(doc):
  keyword_list = []
  keyword_string = keywords(doc)
  keyword_list = keyword_string.split('\n')

  return keyword_list;


comic['Summary_Keywords'] = comic['Summary_final'].apply(lambda x:extract_keywords(x))

In [138]:
def keyword_3(keyword_list):
  keyword_list_3 = keyword_list[0:3]

  return keyword_list_3

comic['Summary_Keywords_3'] = comic['Summary_Keywords'].apply(lambda x:keyword_3(x))

comic.head(3)

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Lower_Summary,No_Contraction_Summary,Clean_Summary,Tokenied_Summary,No_Stopwords_Summary,Summary_final,Summary_Keywords,Summary_Keywords_3
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,"she's young, single and about to achieve her d...","she is young, single and about to achieve her ...",she is young single and about to achieve her ...,"[she, is, young, single, and, about, to, achie...","[young, single, achieve, dream, creating, incr...",young single achieve dream creating incredible...,"[game, gaming, popular streamer, life, single,...","[game, gaming, popular streamer]"
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,"after binge-watching beauty videos online, a s...","after binge-watching beauty videos online, a s...",after binge watching beauty videos online a s...,"[after, binge, watching, beauty, videos, onlin...","[binge, watching, beauty, videos, online, shy,...",binge watching beauty videos online shy comic ...,"[secret, lived, standing, watching, prettiest ...","[secret, lived, standing]"
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,"[after, making, a, grisly, discovery, in, the,...","[making, grisly, discovery, countryside, small...",making grisly discovery countryside small town...,"[takes, underworld, intimidating, grisly]","[takes, underworld, intimidating]"


In [139]:
comic_df = comic[['id','Name', 'Writer', 'Genre', 'Rating',
                 'Summary_Keywords_3']]
comic_df.head(1)

,id,Name,Writer,Genre,Rating,Summary_Keywords_3
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,"[game, gaming, popular streamer]"


In [140]:
from sklearn.feature_extraction.text import CountVectorizer

comic_df['Summary_literal'] = comic_df['Summary_Keywords_3'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
Summary_mat = count_vect.fit_transform(comic_df['Summary_literal'])
print(Summary_mat.shape)

(569, 2356)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [141]:
# 569개의 레코드, 2356개의 개별 단어 피처

In [142]:
from sklearn.metrics.pairwise import cosine_similarity

Summary_sim = cosine_similarity(Summary_mat, Summary_mat)
print(Summary_sim.shape)
print(Summary_sim[:2])

(569, 569)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [143]:
Summary_sim_sorted_ind = Summary_sim.argsort()[:, ::-1]
print(Summary_sim_sorted_ind[:1])

[[  0 248 415 213 264  58 410 260 187 191 190 189 188 185 186 184 183 182
  181 180 179 192 195 193 194 177 196 197 198 199 200 201 202 203 204 205
  206 207 178 174 176 151 157 156 155 154 153 152 150 175 149 148 147 146
  145 144 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
  209 208 212 210 255 262 261 259 258 257 256 254 211 253 252 251 250 249
  247 263 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280
  246 245 244 227 142 214 215 216 217 218 219 220 221 222 223 224 225 226
  228 243 229 230 231 232 233 234 235 236 237 238 239 240 241 242 143 141
  282  43  49  48  47  46  45  44  42  51  41  40  39  38  37  36  50  52
   70  62  68  67  66  65  64  63  61  53  60  59  57  56  55  54  35  34
   33   8  14  13  12  11  10   9   7  32   6   5   4   3   2   1  15  16
   17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  69  71 140
  115 121 120 119 118 117 116 114 123 113 112 111 110 109 108 122 124  72
  133 139 138 137 136 135 134 132 125 

In [145]:
def find_sim_comic(df, sorted_ind, c_name, top_n=10):
    comic_name = df[df['Name'] == c_name]
    comic_index = comic_name.index.values
    similar_indexes = sorted_ind[comic_index, :(top_n*5)]
    similar_indexes = similar_indexes.reshape(-1)
    print(similar_indexes)
    return df.iloc[similar_indexes][:top_n]
    # 마지막 대신 이거 쓰면 return df.iloc[similar_indexes].sort_values('Rating', ascending=False)[:top_n] 평점순으로

In [146]:
# 'Death of a Pop Star'와 비슷한 키워드 가진 웹툰 10개 추천
similar_comic = find_sim_comic(comic_df, Summary_sim_sorted_ind, 'Death of a Pop Star',10)
similar_comic[['Name', 'Rating' , 'Summary_Keywords_3']]

[ 30 409   2 104 101 568 187 192 191 190 189 188 185 186 194 184 183 182
 181 193 195 179 204 210 209 208 207 206 205 203 196 202 201 200 199 198
 197 180 177 178 153 159 158 157 156 155 154 152 212 151]


,Name,Rating,Summary_Keywords_3
30,Death of a Pop Star,9.34,"[takes, lucky, reaper]"
409,The Boxer,9.85,"[fight, takes]"
2,Midnight Poppy Land,9.81,"[takes, underworld, intimidating]"
104,Oh! Holy,9.70,"[invisible, takes, school]"
101,Gourmet Hound,9.82,"[taste, loved years lucky accident, woman]"
568,ã„´ã…‡ã„¹ã…‡ã„´ã„¹,8.83,[team]
187,Omniscient Reader,9.90,"[novel, dokja, world]"
192,Men of the Harem,9.70,"[father, harem, consort]"
191,See You in My 19th Life,9.89,"[life, past lives, ban]"
190,Like Wind on a Dry Branch,9.79,"[empire, taken, plague]"
